# Point Data/Scatter Plot

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import random
from jupyter_dash import JupyterDash

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

### Prepare the Data
Note I added a tooltip. It works the same way as boundaries

In [ ]:
df = pd.read_csv('data/uscities.csv')  # https://simplemaps.com/data/us-cities
cities = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))

statecities = cities[cities.state_id == 'NJ'].copy(deep=True)
statecities['tooltip'] = statecities.city

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
from dash import html
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.javascript import assign

### Exercise: Basic Map Scatterplot

In [ ]:
point_to_layer = None

In [ ]:
geojson = json.loads(statecities.to_json())
geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
cities = dl.GeoJSON(data=geobuf,
                    id="geojson",
                    format="geobuf",
                    options=dict(pointToLayer=point_to_layer))
# Create the app.
app = JupyterDash()
app.layout = html.Div(
    [dl.Map(children=[dl.TileLayer(), cities], center=[40.5, -73], zoom=7)],
    style={
        'width': '1000px',
        'height': '500px'
    },
    id="map")
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Exercise 2: Setting Up Scatterplot with colorbar

In [ ]:
colorscale = ['red', 'yellow', 'green', 'blue', 'purple']  # rainbow
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"  # js lib used for colors
color_prop = 'density'

vmax = df[color_prop].max()
colorbar = dl.Colorbar(colorscale=colorscale,
                       width=20,
                       height=150,
                       min=0,
                       max=vmax,
                       unit='/km2')

point_to_layer = assign("""function(feature, latlng, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);  // chroma lib to construct colorscale
    circleOptions.fillColor = csc(feature.properties[colorProp]);  // set color based on color prop.
    return L.circleMarker(latlng, circleOptions);  // sender a simple circle marker.
}""")

In [ ]:
cities = dl.GeoJSON(data=geobuf,
                    id="geojson",
                    format="geobuf",
                    options=dict(pointToLayer=point_to_layer),
                    hideout=dict(colorProp=color_prop,
                                 circleOptions=dict(fillOpacity=1,
                                                    stroke=False,
                                                    radius=3),
                                 min=0,
                                 max=vmax,
                                 colorscale=colorscale))
# Create the app.
app = JupyterDash(external_scripts=[chroma])
app.layout = html.Div([
    dl.Map(children=[dl.TileLayer(), cities, colorbar],
           center=[40.5, -73],
           zoom=7)
],
                      style={
                          'width': '1000px',
                          'height': '500px'
                      },
                      id="map")
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))